In [ ]:
!pip install pyspark


----
### INCOME

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("landingtoformatted").getOrCreate()

#change the distribucio o  with the accent.
landing_zone = 'landing_zone'
formatted_zone = 'formatted_zone'

datasets = [
    "landing_zone/income/2014_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2015_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2016_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2017_Distribucio_territorial_renda_familiar.csv"
]
income_df = spark.read.csv(datasets[0], header=True, inferSchema=True,nullValue ="-")

for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True, nullValue ="-")
    income_df = income_df.union(df_next)

25/06/24 17:20:44 WARN Utils: Your hostname, Mels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.60.172.110 instead (on interface en0)
25/06/24 17:20:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/24 17:20:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/24 17:20:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
income_df.show(100)

+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
| Any|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Població|Índex RFD Barcelona = 100|
+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
|2014|             1|       Ciutat Vella|         1|            el Raval|   48471|                     65.9|
|2014|             1|       Ciutat Vella|         2|      el Barri Gòtic|   15911|                     98.5|
|2014|             1|       Ciutat Vella|         3|      la Barceloneta|   15181|                     84.5|
|2014|             1|       Ciutat Vella|         4|Sant Pere, Santa ...|   22674|                     92.5|
|2014|             2|           Eixample|         5|       el Fort Pienc|   31785|                    104.5|
|2014|             2|           Eixample|         6|  la Sagrada Família|   51562|                     92.4|
|2014|             

Standarize, eliminate the NULL columns

In [3]:
#we filter the data to remove rows with null values in the 'Índex RFD Barcelona = 100' column or the Names
from pyspark.sql.functions import col

filt_income = income_df.filter(
    ~(
        col('`Índex RFD Barcelona = 100`').isNull() |
        (col('Nom_districte') == 'No consta')
     )
)

----
### Incidencies

In [4]:
datasets = [
    "landing_zone/incidences/2014_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2015_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2016_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2017_Peticions_ciutadanes.csv"
]
incidences_df = spark.read.csv(datasets[0], header=True, inferSchema=True)
for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True)
    incidences_df = incidences_df.union(df_next)

In [5]:
incidences_df.show(100)

+--------+-----------+--------------------+--------------------+--------------------+-------------+-------------+-------------+------------------+------------------+------------------+--------------+-------------------+----------+--------------------+-------------+---------+--------------------+------+------------+------------+-----------+-----------+--------------------+---------------+
|FITXA_ID|      TIPUS|                AREA|             ELEMENT|              DETALL|DIA_DATA_ALTA|MES_DATA_ALTA|ANY_DATA_ALTA|DIA_DATA_TANCAMENT|MES_DATA_TANCAMENT|ANY_DATA_TANCAMENT|CODI_DISTRICTE|          DISTRICTE|CODI_BARRI|               BARRI|SECCIO_CENSAL|TIPUS_VIA|              CARRER|NUMERO|COORDENADA_X|COORDENADA_Y|   LONGITUD|    LATITUD|              SUPORT|CANALS_RESPOSTA|
+--------+-----------+--------------------+--------------------+--------------------+-------------+-------------+-------------+------------------+------------------+------------------+--------------+-------------------

In [5]:
from pyspark.sql.functions import col, sum
incidences_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in incidences_df.columns]).show()

+--------+-----+----+-------+------+-------------+-------------+-------------+------------------+------------------+------------------+--------------+---------+----------+------+-------------+---------+------+------+------------+------------+--------+-------+------+---------------+
|FITXA_ID|TIPUS|AREA|ELEMENT|DETALL|DIA_DATA_ALTA|MES_DATA_ALTA|ANY_DATA_ALTA|DIA_DATA_TANCAMENT|MES_DATA_TANCAMENT|ANY_DATA_TANCAMENT|CODI_DISTRICTE|DISTRICTE|CODI_BARRI| BARRI|SECCIO_CENSAL|TIPUS_VIA|CARRER|NUMERO|COORDENADA_X|COORDENADA_Y|LONGITUD|LATITUD|SUPORT|CANALS_RESPOSTA|
+--------+-----+----+-------+------+-------------+-------------+-------------+------------------+------------------+------------------+--------------+---------+----------+------+-------------+---------+------+------+------------+------------+--------+-------+------+---------------+
|       0|    0|   0|      0|     0|            0|            0|            0|                 0|                 0|                 0|        250716| 

In [6]:
from pyspark.sql.functions import col

incidences_df = incidences_df.withColumn("CODI_BARRI", col("CODI_BARRI").cast("string"))


We filter NULL values


In [7]:
from pyspark.sql.functions import when

incidences_cleaned = incidences_df.withColumn(
    "CODI_BARRI", when(col("CODI_BARRI") == 'NULL', None).otherwise(col("CODI_BARRI"))
).withColumn(
    "BARRI", when(col("BARRI") == 'NULL', None).otherwise(col("BARRI"))
)
filt_incidences = incidences_cleaned.filter(
    col("CODI_BARRI").isNotNull() | col("BARRI").isNotNull()
)

In [8]:
# we need to rename the columns to match the income_df
# We change the column names ANY_DATA_ALTA, to ANY so we can match later the year where the incident happened.

filt_incidences = (filt_incidences.withColumnRenamed('CODI_BARRI', 'Codi_Barri')
        .withColumnRenamed('CODI_DISTRICTE', 'Codi_Districte').withColumnRenamed('ANY_DATA_ALTA', 'Any'))

with renaming the columns we facilitate the later the unification of the tables

In [10]:
filt_incidences.show(100)

+--------+-----------+--------------------+--------------------+--------------------+-------------+-------------+----+------------------+------------------+------------------+--------------+-------------------+----------+--------------------+-------------+----------+--------------------+------+------------+------------+-----------+-----------+---------+---------------+
|FITXA_ID|      TIPUS|                AREA|             ELEMENT|              DETALL|DIA_DATA_ALTA|MES_DATA_ALTA| Any|DIA_DATA_TANCAMENT|MES_DATA_TANCAMENT|ANY_DATA_TANCAMENT|Codi_Districte|          DISTRICTE|Codi_Barri|               BARRI|SECCIO_CENSAL| TIPUS_VIA|              CARRER|NUMERO|COORDENADA_X|COORDENADA_Y|   LONGITUD|    LATITUD|   SUPORT|CANALS_RESPOSTA|
+--------+-----------+--------------------+--------------------+--------------------+-------------+-------------+----+------------------+------------------+------------------+--------------+-------------------+----------+--------------------+-------------+

In [9]:
filt_incidences.groupBy("Codi_Barri").count().orderBy("count", ascending=False).show()

+----------+-----+
|Codi_Barri|count|
+----------+-----+
|        31|11510|
|         7|10915|
|         1|10860|
|        18| 9409|
|        60| 9278|
|        68| 9245|
|         4| 8502|
|         9| 8379|
|        26| 8265|
|        11| 8249|
|        19| 8240|
|        35| 7498|
|         2| 7385|
|         8| 7211|
|         6| 6575|
|        37| 6505|
|        10| 6423|
|        43| 6309|
|        13| 5721|
|        64| 5299|
+----------+-----+
only showing top 20 rows



-----
### Population

In [10]:
datasets = [
    "landing_zone/population_by_geographical/2014_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2015_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2016_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2017_pad_mdb_nacionalitat-regio_sexe.json"
]
population_df = spark.read.json(datasets[0])
for dataset in datasets[1:]:
    df_next = spark.read.json(dataset)
    population_df = population_df.union(df_next)

In [17]:
population_df.show(100)

+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|Codi_Barri|Codi_Districte|Data_Referencia|NACIONALITAT_REGIO|     Nom_Barri|Nom_Districte|SEXE|Valor|
+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   1|   ..|
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   2|   ..|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   1|    6|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   2|    8|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   1|  682|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   2|  989|
|         1|             1|     2014-01-01|                 4|      el Ra

In [11]:
import pandas as pd
csv = pd.read_csv('landing_zone/pad_dimensions.csv')

In [15]:
csv

,Codi_Dimensio,Desc_Dimensio,Codi_Valor,Desc_Valor_CA,Desc_Valor_ES,Desc_Valor_EN
0,1,SEXE,1,Dona,Mujer,Female
1,1,SEXE,2,Home,Hombre,Male
2,2,EDAT_1,0,0 anys,0 años,0 years
3,2,EDAT_1,1,1 anys,1 años,1 years
4,2,EDAT_1,2,2 anys,2 años,2 years
...,...,...,...,...,...,...
1138,26,NACIONALITAT_DEST,142,Maurici,Mauricio,Mauritius
1139,26,NACIONALITAT_DEST,144,"Iraq, l'",Iraq,Iraq
1140,26,NACIONALITAT_DEST,145,Kenya,Kenia,Kenya
1141,26,NACIONALITAT_DEST,146,Seychelles,Seychelles,Seychelles


In [12]:
population_df.columns

['Codi_Barri',
 'Codi_Districte',
 'Data_Referencia',
 'NACIONALITAT_REGIO',
 'Nom_Barri',
 'Nom_Districte',
 'SEXE',
 'Valor']

In [13]:
csv = spark.createDataFrame(csv)

In [20]:
csv

DataFrame[Codi_Dimensio: bigint, Desc_Dimensio: string, Codi_Valor: bigint, Desc_Valor_CA: string, Desc_Valor_ES: string, Desc_Valor_EN: string]

In [14]:
from pyspark.sql import functions as F

cols_to_map = ['SEXE', 'NACIONALITAT_REGIO']

for col_name in cols_to_map:
    mapping_dict = (csv.filter(F.col('Desc_Dimensio') == col_name)
                      .select('Codi_Valor', 'Desc_Valor_EN')
                      .where(F.col('Codi_Valor').isNotNull())
                      .rdd.map(lambda r: (int(r['Codi_Valor']), r['Desc_Valor_EN']))
                      .collectAsMap())

    expr = None
    for code, label in mapping_dict.items():
        cond = F.col(col_name) == code
        expr = F.when(cond, F.lit(label)) if expr is None else expr.when(cond, F.lit(label))

    expr = expr.otherwise(F.col(col_name).cast("string"))

    population_df = population_df.withColumn(f"{col_name}_desc", expr)

In [15]:
from pyspark.sql import functions as F
population_df = population_df.withColumn("Any", F.substring("Data_Referencia", 1, 4).cast("int"))

In [16]:
dim_names = [row['Desc_Dimensio'] for row in csv.select('Desc_Dimensio').distinct().collect()]

In [17]:
dim_names

['EDAT_1',
 'SEXE',
 'LLOC_NAIX',
 'NIV_EDUCA_esta',
 'NACIONALITAT_REGIO',
 'NACIONALITAT_G',
 'LLOC_NAIX_REGIO',
 'EDAT_Q',
 'LLOC_NAIX_CCAA',
 'DECADA',
 'LLOC_NAIX_PAIS',
 'NACIONALITAT_PAIS',
 'LLOC_NAIX_CONTINENT',
 'TIPUS_DOMICILI',
 'N_PERSONES_AGG',
 'NACIONALITAT_CONTINENT',
 'NACIONALITAT_DOMICILI',
 'DOM_65_99',
 'DOM_HOMES',
 'DOM_DONES',
 'DOM_00_18',
 'DOM_18_64',
 'CODI_BARRI_DEST',
 'CONTINENT_MOV',
 'NACIONALITAT_INI',
 'NACIONALITAT_DEST']

In [18]:
population_df

DataFrame[Codi_Barri: bigint, Codi_Districte: bigint, Data_Referencia: string, NACIONALITAT_REGIO: bigint, Nom_Barri: string, Nom_Districte: string, SEXE: bigint, Valor: string, SEXE_desc: string, NACIONALITAT_REGIO_desc: string, Any: int]

We merged the two datasets so that, instead of having encoded values in the 'SEXE' or 'NACIONALITAT_REGIO' columns, we now have the actual values. To do this, we had to download an additional dataset that contained both the actual values and their corresponding encoding numbers.

In [19]:
population_df.printSchema()          # SEXE_desc y NACIONALITAT_REGIO_desc son string
population_df.select("SEXE", "SEXE_desc",
                     "NACIONALITAT_REGIO", "NACIONALITAT_REGIO_desc").show(10)


root
 |-- Codi_Barri: long (nullable = true)
 |-- Codi_Districte: long (nullable = true)
 |-- Data_Referencia: string (nullable = true)
 |-- NACIONALITAT_REGIO: long (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- SEXE: long (nullable = true)
 |-- Valor: string (nullable = true)
 |-- SEXE_desc: string (nullable = true)
 |-- NACIONALITAT_REGIO_desc: string (nullable = true)
 |-- Any: integer (nullable = true)

+----+---------+------------------+-----------------------+
|SEXE|SEXE_desc|NACIONALITAT_REGIO|NACIONALITAT_REGIO_desc|
+----+---------+------------------+-----------------------+
|   1|   Female|                 1|         Eastern Africa|
|   2|     Male|                 1|         Eastern Africa|
|   1|   Female|                 2|          Middle Africa|
|   2|     Male|                 2|          Middle Africa|
|   1|   Female|                 3|        Northern Africa|
|   2|     Male|                 3|        Nor

In [20]:
population_df.show(100)

+----------+--------------+---------------+------------------+--------------+-------------+----+-----+---------+-----------------------+----+
|Codi_Barri|Codi_Districte|Data_Referencia|NACIONALITAT_REGIO|     Nom_Barri|Nom_Districte|SEXE|Valor|SEXE_desc|NACIONALITAT_REGIO_desc| Any|
+----------+--------------+---------------+------------------+--------------+-------------+----+-----+---------+-----------------------+----+
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   1|   ..|   Female|         Eastern Africa|2014|
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   2|   ..|     Male|         Eastern Africa|2014|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   1|    6|   Female|          Middle Africa|2014|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   2|    8|     Male|          Middle Africa|2014|
|     

In [21]:
from pyspark.sql import functions as F

population_sel = (
    population_df
        .withColumn(
            "Valor_int",
            F.when(F.col("Valor") == "..", None)
             .otherwise(F.col("Valor"))
             .cast("int")
        )
        .select(
            "Codi_Barri",
            "Codi_Districte",
            "Nom_Barri",
            "Nom_Districte",
            "Valor_int",
            "Any",
            "SEXE_desc",
            "NACIONALITAT_REGIO_desc"
        )
)


In [22]:
population_sel =population_sel.filter(F.col("Valor_int").isNotNull())

In [23]:
population_sel.show(100)

+----------+--------------+--------------------+-------------+---------+----+---------+-----------------------+
|Codi_Barri|Codi_Districte|           Nom_Barri|Nom_Districte|Valor_int| Any|SEXE_desc|NACIONALITAT_REGIO_desc|
+----------+--------------+--------------------+-------------+---------+----+---------+-----------------------+
|         1|             1|            el Raval| Ciutat Vella|        6|2014|   Female|          Middle Africa|
|         1|             1|            el Raval| Ciutat Vella|        8|2014|     Male|          Middle Africa|
|         1|             1|            el Raval| Ciutat Vella|      682|2014|   Female|        Northern Africa|
|         1|             1|            el Raval| Ciutat Vella|      989|2014|     Male|        Northern Africa|
|         1|             1|            el Raval| Ciutat Vella|       56|2014|   Female|         Western Africa|
|         1|             1|            el Raval| Ciutat Vella|      124|2014|     Male|         Western 

### Before this we have to partition the tables in the more optimal way for our analysis.


In [24]:
filt_incidences.write\
    .mode('overwrite')\
    .partitionBy("Codi_Barri", "Any")\
    .parquet("formatted_zone/incidences")

filt_income.write\
    .mode('overwrite')\
    .partitionBy("Codi_Barri", "Any")\
    .parquet("formatted_zone/income")

population_sel.write\
    .mode('overwrite')\
    .partitionBy("Codi_Barri", "Any")\
    .parquet("formatted_zone/population_by_geographical")


25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/06/24 17:21:57 WARN MemoryManager: Total allocation exceeds 95.0